In [107]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [9]:
df_cibil_feat = pd.read_csv("./dw_out/df_cibil_feat.csv")

In [10]:
df_cibil_feat=df_cibil_feat.drop(columns=['Unnamed: 0'])
df_cibil_feat.head()

,key,timestamp,datetime_formatted,cibil_score,total_email,gender,age,open_loans,closed_loans,total_address,...,last_3_months_dpd_cc,last_6_months_dpd_cc,last_12_months_dpd_cc,last_36_months_dpd_cc,next_3_months_dpd_cc,next_6_months_dpd_cc,next_12_months_dpd_cc,next_36_months_dpd_cc,time_rank,output
0,0-10,11-16,2016-11-01,645.0,4,0,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0
1,0-9,10-18,2018-10-01,645.0,4,0,38,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0
2,0-8,11-18,2018-11-01,645.0,4,0,38,2,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0
3,3-6,3-18,2018-03-01,757.0,1,0,24,1,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2
4,3-4,9-18,2018-09-01,757.0,1,0,25,3,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0


In [219]:
df = df_cibil_feat[['cibil_score', 'datetime_formatted', 'total_email',
       'gender', 'age', 'open_loans', 'closed_loans', 'total_address','sanc_amount',
        'total_phone_nos', 'total_enq_till_date', 'unique_enquiry_purpose',
       'total_enq_amt',  'loan_type', 'ownership',
       'enquiry_purpose','last_3_months_dpd_gl',
       'last_6_months_dpd_gl', 'last_12_months_dpd_gl',
       'last_36_months_dpd_gl','last_3_months_dpd_personal', 'last_6_months_dpd_personal',
       'last_12_months_dpd_personal', 'last_36_months_dpd_personal','last_3_months_dpd_cvl', 'last_6_months_dpd_cvl',
       'last_12_months_dpd_cvl', 'last_36_months_dpd_cvl','last_3_months_dpd_cc', 'last_6_months_dpd_cc', 'last_12_months_dpd_cc',
       'last_36_months_dpd_cc',
       'total_enq_till_date_on_user_id_enq_purpose',
       'unique_enquiry_purpose_on_user_id_enq_purpose',
       'total_enq_amt_on_user_id_enq_purpose','output']]

In [220]:
df.loan_type.value_counts()

3    749
0    744
4    354
5    304
2    233
1     29
Name: loan_type, dtype: int64

In [221]:
df = df[df['loan_type'].isin([2,4,5])]

In [222]:
df.shape

(891, 36)

In [223]:
import numpy as np

np.sort(df.datetime_formatted.unique())

array(['2005-01-01', '2005-10-01', '2006-07-01', '2006-11-01',
       '2007-06-01', '2008-01-01', '2008-04-01', '2010-11-01',
       '2011-07-01', '2011-08-01', '2011-10-01', '2012-04-01',
       '2013-01-01', '2013-04-01', '2013-08-01', '2013-10-01',
       '2013-12-01', '2014-01-01', '2014-02-01', '2014-03-01',
       '2014-04-01', '2014-05-01', '2014-06-01', '2014-07-01',
       '2014-10-01', '2014-11-01', '2014-12-01', '2015-01-01',
       '2015-02-01', '2015-06-01', '2015-07-01', '2015-08-01',
       '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
       '2016-01-01', '2016-02-01', '2016-04-01', '2016-05-01',
       '2016-06-01', '2016-07-01', '2016-08-01', '2016-10-01',
       '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01',
       '2017-03-01', '2017-04-01', '2017-05-01', '2017-08-01',
       '2017-09-01', '2017-11-01', '2017-12-01', '2018-01-01',
       '2018-02-01', '2018-04-01', '2018-05-01', '2018-06-01',
       '2018-07-01', '2018-08-01', '2018-09-01', '2018-

In [224]:
print(len(df.datetime_formatted.unique()))

111


In [225]:
0.20*164

32.800000000000004

In [294]:
df.shape

(891, 36)

In [295]:
df['output'].value_counts()

0    738
2    116
1     37
Name: output, dtype: int64

In [296]:
df['output'] = df.apply(lambda s: s['output'] if s['output'] <=0 else 1,axis=1)

In [297]:
df['output'].value_counts()

0    738
1    153
Name: output, dtype: int64

In [470]:
df_train = df[df['datetime_formatted'] <= '2021-08-01']
df_valid = df[df['datetime_formatted'] > '2021-08-01']

In [471]:
df_valid['output'].value_counts()

0    101
1      4
Name: output, dtype: int64

In [472]:
df_train['output'].value_counts()

0    637
1    149
Name: output, dtype: int64

In [ ]:
# df_train_0 = df_train[df_train['output']==0]
# df_train_1 = df_train[df_train['output']==1]
# df_train_0 = df_train_0.sample(n=149, random_state=42)
# df_train_sample = df_train_0.append(df_train_1)
# df_train_sample.shape

In [498]:
df_train_0 = df_train[df_train['output']==0]
df_train_1 = df_train[df_train['output']==1]

In [499]:
df_train_0.shape, df_train_1.shape

((637, 36), (149, 36))

In [500]:
df_train_0 = df_train_0.sample(n=149, random_state=42)
df_train_0.shape

(149, 36)

In [501]:
df_train_sample = df_train_0.append(df_train_1)
df_train_sample.shape

(298, 36)

In [524]:
X_train = df_train_sample[['total_email',
       'gender', 'age', 'open_loans', 'closed_loans', 'total_address','sanc_amount',
        'total_phone_nos',
        'loan_type', 'ownership',
        'last_3_months_dpd_gl',
       'last_6_months_dpd_gl', 'last_12_months_dpd_gl',
       'last_36_months_dpd_gl','last_3_months_dpd_personal', 'last_6_months_dpd_personal',
       'last_12_months_dpd_personal', 'last_36_months_dpd_personal','last_3_months_dpd_cvl', 'last_6_months_dpd_cvl',
       'last_12_months_dpd_cvl', 'last_36_months_dpd_cvl','last_3_months_dpd_cc', 'last_6_months_dpd_cc', 'last_12_months_dpd_cc',
       'last_36_months_dpd_cc']]
       #'total_enq_till_date', 'unique_enquiry_purpose',
       #'total_enq_amt',  
       #'enquiry_purpose',
       #'total_enq_till_date_on_user_id_enq_purpose',
       #'unique_enquiry_purpose_on_user_id_enq_purpose',
       #'total_enq_amt_on_user_id_enq_purpose']]

y_train = df_train_sample[['output']]

In [525]:
X_test = df_valid[['total_email',
       'gender', 'age', 'open_loans', 'closed_loans', 'total_address','sanc_amount',
        'total_phone_nos', 
        'loan_type', 'ownership',
        'last_3_months_dpd_gl',
       'last_6_months_dpd_gl', 'last_12_months_dpd_gl',
       'last_36_months_dpd_gl','last_3_months_dpd_personal', 'last_6_months_dpd_personal',
       'last_12_months_dpd_personal', 'last_36_months_dpd_personal','last_3_months_dpd_cvl', 'last_6_months_dpd_cvl',
       'last_12_months_dpd_cvl', 'last_36_months_dpd_cvl','last_3_months_dpd_cc', 'last_6_months_dpd_cc', 'last_12_months_dpd_cc',
       'last_36_months_dpd_cc']]
       #'total_enq_till_date_on_user_id_enq_purpose',
       #'total_enq_till_date_on_user_id_enq_purpose',
       #'unique_enquiry_purpose_on_user_id_enq_purpose',
       #'total_enq_amt_on_user_id_enq_purpose']]

y_test = df_valid[['output']]

In [526]:
from sklearn.metrics import DistanceMetric


In [527]:
y = df[['output']]
y['output'].value_counts()

0    738
1    153
Name: output, dtype: int64

In [528]:
y_train['output'].value_counts()

0    149
1    149
Name: output, dtype: int64

In [529]:
y_test['output'].value_counts()

0    101
1      4
Name: output, dtype: int64

In [530]:
.20*df.shape[0]

178.20000000000002

In [531]:
X_train.shape, X_test.shape

((298, 26), (105, 26))

In [532]:
y_train.shape, y_test.shape

((298, 1), (105, 1))

In [533]:
xgb_classifier =XGBClassifier(n_estimators=1000)

In [534]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train['output'] #provide your own target name
)
# 
# xgb_classifier.fit(X, y, sample_weight=sample_weights)

In [535]:
len(sample_weights)

298

In [536]:
xgb_classifier.fit(X_train,y_train,sample_weight=sample_weights)
#xgb_classifier.fit(X_train,y_train)


/home/ubuntu/anaconda3/envs/faiss/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ubuntu/anaconda3/envs/faiss/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ubuntu/anaconda3/envs/faiss/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example 

[10:43:46] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=2,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [537]:
predictions = xgb_classifier.predict(X_test)


In [538]:
y_proba = xgb_classifier.predict_proba(X_test)

In [539]:
print("Accuracy of Model::",accuracy_score(y_test,predictions))


Accuracy of Model:: 0.7333333333333333


In [540]:
#print("Accuracy of Model::",accuracy_score(y_test,predictions))


In [541]:
y_test['output'].value_counts()

0    101
1      4
Name: output, dtype: int64

In [542]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve,classification_report

def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """

    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

In [543]:
print(confusion_matrix(y_test,predictions))

[[75 26]
 [ 2  2]]


In [544]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      0.74      0.84       101
           1       0.07      0.50      0.12         4

    accuracy                           0.73       105
   macro avg       0.52      0.62      0.48       105
weighted avg       0.94      0.73      0.82       105



In [545]:
import pickle
file_name = "./dw_out/xgb_cibil_classifier.pkl"

# save
pickle.dump(xgb_classifier, open(file_name, "wb"))
